# Database structure

## Docs
- [Introduction to the Python Driver for Oracle Database](https://python-oracledb.readthedocs.io/en/latest/user_guide/introduction.html#introduction-to-the-python-driver-for-oracle-database)
- [Python python-oracledb Driver](https://oracle.github.io/python-oracledb/)
- [Python: Read Data from Oracle Database](https://kontext.tech/article/1019/python-read-data-from-oracle-database)

## Prepare  connection to the database

In [1]:
    import oracledb
    import getpass
    
    username = "CUSTDOC"
    userpwd = getpass.getpass("Enter password: ")

    host="localhost"
    port="1521"
    service_name="XEPDB1"
    dsn = f'{username}/{userpwd}@{host}:{port}/{service_name}'
   

Enter password:  ········


## Service Functions to build readable Table Structure

In [5]:
def read_table_str( cursor, a_owner, a_table_name):
    sql_query_t = """
    SELECT d.OWNER as TAB_OWNER,
           d.TABLE_NAME as TAB_NAME ,
           d.TABLE_TYPE as TAB_TYPE,
           d.COMMENTS   as TAB_REM
           FROM All_Tab_Comments  d
           WHERE d.owner=:OWNER and  d.TABLE_NAME like :TABLE_NAME
     """
    t_name=a_table_name
    t_owner=a_owner
    print("TABLE")
    cursor.execute(sql_query_t, { "OWNER": t_owner, "TABLE_NAME": t_name})
    column_names = [col[0] for col in cursor.description]
    rows = cursor.fetchall()
    if rows:
        df = pd.DataFrame(rows, columns=column_names)
        display(df)
    else:
        print("Запит не повернув жодного рядка.")
        
def read_columns_str( cursor, a_owner, a_table_name):   
    sql_query_c = """
    SELECT d.OWNER       as TAB_OWNER,
           d.TABLE_NAME  as TAB_NAME,
           d.COLUMN_NAME as COL_NAME,
           d.DATA_TYPE   as COL_TYPE,
           DECODE(d.DATA_PRECISION,null, d.DATA_LENGTH, d.DATA_PRECISION) as COL_LENGTH,
           d.DATA_SCALE as COL_DEC,
           C.comments       as COL_REM
           FROM all_tab_columns d, All_COL_Comments  C
            WHERE d.owner=:OWNER and   d.TABLE_NAME like :TABLE_NAME and
                  d.owner=c.owner
              and d.TABLE_NAME=c.TABLE_NAME
              and d.COLUMN_NAME=c.COLUMN_NAME               
           ORDER BY d.TABLE_NAME, d.COLUMN_ID
     """
    t_name=a_table_name
    t_owner=a_owner
    print("TABLE COLUMS")
    cursor.execute(sql_query_c, { "OWNER": t_owner, "TABLE_NAME": t_name})
    column_names = [col[0] for col in cursor.description]
    rows = cursor.fetchall()
    if rows:
        df = pd.DataFrame(rows, columns=column_names)
        display(df)
    else:
        print("Запит не повернув жодного рядка.")       

def read_constr_c( cursor, a_owner, a_table_name):   
    sql_query_c = """
        SELECT TB.OWNER            AS TAB_OWNER,
               TB.TABLE_NAME       AS TAB_NAME,
               TC.CONSTRAINT_NAME  AS CNST_NAME,
               TC.SEARCH_CONDITION AS CNST_VALUE
          FROM SYS.ALL_TAB_COMMENTS TB,
             SYS.ALL_CONSTRAINTS TC
             WHERE 
             TB.OWNER=:OWNER AND   TB.TABLE_NAME LIKE :TABLE_NAME AND 
            (TC.OWNER = TB.OWNER) AND
            (TC.TABLE_NAME = TB.TABLE_NAME) AND
           (TC.CONSTRAINT_TYPE = 'C')
     """
    t_name=a_table_name
    t_owner=a_owner
    print("TABLE CONSTRAINTS")
    cursor.execute(sql_query_c, { "OWNER": t_owner, "TABLE_NAME": t_name})
    column_names = [col[0] for col in cursor.description]
    rows = cursor.fetchall()
    if rows:
        df = pd.DataFrame(rows, columns=column_names)
        display(df)
    else:
        print("Запит не повернув жодного рядка.")       
def read_constr_p( cursor, a_owner, a_table_name):   
    sql_query_p = """
        SELECT TB.OWNER AS TAB_OWNER, TB.TABLE_NAME AS TAB_NAME, TC.CONSTRAINT_NAME AS CNST_NAME, 
        TC.CONSTRAINT_TYPE AS CNST_TYPE, CC.COLUMN_NAME AS COL_NAME, TO_CHAR(CC.COLUMN_POSITION, '999') AS COL_POS 
        FROM SYS.ALL_TAB_COMMENTS TB, SYS.ALL_CONSTRAINTS TC, SYS.ALL_IND_COLUMNS CC 
        WHERE TB.OWNER=:OWNER AND TB.TABLE_NAME LIKE :TABLE_NAME
        AND (TC.OWNER = TB.OWNER) AND
        (TC.TABLE_NAME = TB.TABLE_NAME) AND
        (TC.CONSTRAINT_TYPE IN ('P', 'U') ) AND
        ( TB.OWNER=CC.TABLE_OWNER ) AND
        ( TB.TABLE_NAME=CC.TABLE_NAME ) AND
        ( TC.INDEX_NAME=CC.INDEX_NAME )
        ORDER BY DECODE(TC.CONSTRAINT_TYPE,'P',1,2), TC.CONSTRAINT_NAME, TC.CONSTRAINT_TYPE, CC.COLUMN_POSITION
     """
    t_name=a_table_name
    t_owner=a_owner
    print("TABLE PRIMARY KEY CONSTRAINT")
    cursor.execute(sql_query_p, { "OWNER": t_owner, "TABLE_NAME": t_name})
    column_names = [col[0] for col in cursor.description]
    rows = cursor.fetchall()
    if rows:
        df = pd.DataFrame(rows, columns=column_names)
        display(df)
    else:
        print("Запит не повернув жодного рядка.")   

def read_constr_fk( cursor, a_owner, a_table_name):   
    sql_query_fk = """
        SELECT decode(cfk.position,1,Tc.OWNER)            AS TAB_OWNER,
               decode(cfk.position,1,Tc.TABLE_NAME)       AS TAB_NAME,
               decode(cfk.position,1,TC.CONSTRAINT_NAME)  AS CNST_NAME,
               Cfk.column_name                        AS FK_COL_NAME,
               decode(cfk.position,1,cr.owner)            AS REF_TAB_OWNER,
               decode(cfk.position,1,cr.TABLE_NAME)       AS REF_TAB_NAME,
               CR.COLUMN_NAME                             as FK_COL_NAME
          FROM SYS.ALL_CONSTRAINTS TC 
              inner join sys.all_cons_columns cfk 
                 on cfk.owner = tc.owner 
                 and cfk.constraint_name = tc.constraint_name 
              inner join sys.all_cons_columns cr 
                 on cr.owner = tc.r_owner 
                 and cr.constraint_name = tc.r_constraint_name 
           WHERE 
            TC.OWNER=:OWNER AND   TC.TABLE_NAME LIKE :TABLE_NAME AND 
            TC.CONSTRAINT_TYPE in ( 'R')
            and cr.POSITION=cfk.POSITION
           order by tc.TABLE_NAME,TC.CONSTRAINT_NAME,cfk.position
     """
    t_name=a_table_name
    t_owner=a_owner
    print("TABLE FORAINGN KEY CONSTRAINT")
    cursor.execute(sql_query_fk, { "OWNER": t_owner, "TABLE_NAME": t_name})
    column_names = [col[0] for col in cursor.description]
    rows = cursor.fetchall()
    if rows:
        df = pd.DataFrame(rows, columns=column_names)
        display(df)
    else:
        print("Запит не повернув жодного рядка.")              


## Table List

In [6]:
import pandas as pd
from IPython.display import display


# SQL- запит як багаторядкова змінна
sql_query = """
   SELECT T.OWNER, T.TABLE_NAME, C.TABLE_TYPE, C.COMMENTS
   FROM ALL_TABLES T
   LEFT OUTER JOIN ALL_TAB_COMMENTS C ON C.OWNER=T.OWNER AND C.TABLE_NAME=T.TABLE_NAME 
   WHERE T.OWNER ='CUSTDOC'
 """

connection = None
cursor = None
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення

try:
    connection = oracledb.connect(dsn=dsn)
    cursor = connection.cursor()
    cursor.execute(sql_query)
    column_names = [col[0] for col in cursor.description]
    rows = cursor.fetchall()
    if rows:
        df = pd.DataFrame(rows, columns=column_names)
        display(df)
        print(f"Fetched rows={len(rows)}")
    else:
        pint("Запит не повернув жодного рядка.")
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Помилка Oracle: {error_obj.code} - {error_obj.message}")
finally:
    if cursor:
        cursor.close()
        print("\nКурсор закрито.")
    if connection:
        connection.close()
        print("З'єднання з БД закрито.")

,OWNER,TABLE_NAME,TABLE_TYPE,COMMENTS
0,CUSTDOC,CUST$ARTIST,TABLE,Famous Artists
1,CUSTDOC,CUST$CUST,TABLE,Customer Records
2,CUSTDOC,CUST$D$DOCTYPE,TABLE,Types Customer Documents
3,CUSTDOC,CUST$DOCS,TABLE,Customer Documents
4,CUSTDOC,CUST$DOCS$ATTACH,TABLE,Customer Documents: Attached files
5,CUSTDOC,CUST$DOCS$URLS,TABLE,Customer Documents Attached files


Fetched rows=6

Курсор закрито.
З'єднання з БД закрито.


## TABLE ```CUSTDOC.CUST$ARTIST``` 

In [9]:
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення
connection = None
cursor = None
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення
try:
    connection = oracledb.connect(dsn=dsn)
    cursor = connection.cursor()
    read_table_str( cursor, 'CUSTDOC', 'CUST$ARTIST')
    read_columns_str( cursor, 'CUSTDOC', 'CUST$ARTIST')
    read_constr_c( cursor, 'CUSTDOC', 'CUST$ARTIST')
    read_constr_p( cursor, 'CUSTDOC', 'CUST$ARTIST')
    read_constr_fk( cursor, 'CUSTDOC', 'CUST$ARTIST')
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Помилка Oracle: {error_obj.code} - {error_obj.message}")
finally:
    if cursor:
        cursor.close()
        print("\nКурсор закрито.")
    if connection:
        connection.close()
        print("З'єднання з БД закрито.")

TABLE


,TAB_OWNER,TAB_NAME,TAB_TYPE,TAB_REM
0,CUSTDOC,CUST$ARTIST,TABLE,Famous Artists


TABLE COLUMS


,TAB_OWNER,TAB_NAME,COL_NAME,COL_TYPE,COL_LENGTH,COL_DEC,COL_REM
0,CUSTDOC,CUST$ARTIST,IDA,NUMBER,22,0.0,Record IDentifier
1,CUSTDOC,CUST$ARTIST,ARTIST_EN,VARCHAR2,64,NaN,Artist Name in English
2,CUSTDOC,CUST$ARTIST,ARTIST_UA,VARCHAR2,64,NaN,Artist Name in Ukrainian
3,CUSTDOC,CUST$ARTIST,PHOTO_URL,VARCHAR2,160,NaN,Artist Photo URL
4,CUSTDOC,CUST$ARTIST,WORK_URL,VARCHAR2,160,NaN,Artist Work URL
5,CUSTDOC,CUST$ARTIST,WORK_TITLE_EN,VARCHAR2,160,NaN,Artist Work Title in English
6,CUSTDOC,CUST$ARTIST,WORK_TITLE_UA,VARCHAR2,160,NaN,Artist Work Title in Ukrainian
7,CUSTDOC,CUST$ARTIST,IUSRNM,VARCHAR2,30,NaN,Who Inserted the record
8,CUSTDOC,CUST$ARTIST,IDT,DATE,7,NaN,Date and Time of Insertion
9,CUSTDOC,CUST$ARTIST,MUSRNM,VARCHAR2,30,NaN,Who Modified the record


TABLE CONSTRAINTS


,TAB_OWNER,TAB_NAME,CNST_NAME,CNST_VALUE
0,CUSTDOC,CUST$ARTIST,SYS_C008557,"""IDA"" IS NOT NULL"


TABLE PRIMARY KEY CONSTRAINT


,TAB_OWNER,TAB_NAME,CNST_NAME,CNST_TYPE,COL_NAME,COL_POS
0,CUSTDOC,CUST$ARTIST,CUST$ARTIST_PK,P,IDA,1


TABLE FORAINGN KEY CONSTRAINT
Запит не повернув жодного рядка.

Курсор закрито.
З'єднання з БД закрито.


## TABLE ```CUSTDOC.CUST$CUST```

In [10]:
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення
connection = None
cursor = None
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення
try:
    connection = oracledb.connect(dsn=dsn)
    cursor = connection.cursor()
    read_table_str( cursor, 'CUSTDOC', 'CUST$CUST')
    read_columns_str( cursor, 'CUSTDOC', 'CUST$CUST')
    read_constr_c( cursor, 'CUSTDOC', 'CUST$CUST')
    read_constr_p( cursor, 'CUSTDOC', 'CUST$CUST')
    read_constr_fk( cursor, 'CUSTDOC', 'CUST$CUST')
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Помилка Oracle: {error_obj.code} - {error_obj.message}")
finally:
    if cursor:
        cursor.close()
        print("\nКурсор закрито.")
    if connection:
        connection.close()
        print("З'єднання з БД закрито.")

TABLE


,TAB_OWNER,TAB_NAME,TAB_TYPE,TAB_REM
0,CUSTDOC,CUST$CUST,TABLE,Customer Records


TABLE COLUMS


,TAB_OWNER,TAB_NAME,COL_NAME,COL_TYPE,COL_LENGTH,COL_DEC,COL_REM
0,CUSTDOC,CUST$CUST,CUSTID,VARCHAR2,9,None,CUSTOMER IDentifier
1,CUSTDOC,CUST$CUST,CUSTTYPE,VARCHAR2,2,None,"CUSTOMER TYPE ( CI - Individuals, LE -LEgal En..."
2,CUSTDOC,CUST$CUST,CUSTNAME,VARCHAR2,180,None,Customer name
3,CUSTDOC,CUST$CUST,IUSRNM,VARCHAR2,30,None,Who Inserted the record
4,CUSTDOC,CUST$CUST,IDT,DATE,7,None,Date and Time of Insertion
5,CUSTDOC,CUST$CUST,MUSRNM,VARCHAR2,30,None,Who Modified the record
6,CUSTDOC,CUST$CUST,MDT,DATE,7,None,Date and Time of Modification


TABLE CONSTRAINTS


,TAB_OWNER,TAB_NAME,CNST_NAME,CNST_VALUE
0,CUSTDOC,CUST$CUST,CUST$CUST_N,CUSTNAME IS NOT NULL
1,CUSTDOC,CUST$CUST,CUST$CUST_T,"CUSTTYPE IN ('CI', 'LE')"


TABLE PRIMARY KEY CONSTRAINT


,TAB_OWNER,TAB_NAME,CNST_NAME,CNST_TYPE,COL_NAME,COL_POS
0,CUSTDOC,CUST$CUST,CUST$CUST_PK,P,CUSTID,1


TABLE FORAINGN KEY CONSTRAINT
Запит не повернув жодного рядка.

Курсор закрито.
З'єднання з БД закрито.


## TABLE ```CUSTDOC.CUST$D$DOCTYPE```


In [11]:
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення
connection = None
cursor = None
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення
try:
    connection = oracledb.connect(dsn=dsn)
    cursor = connection.cursor()
    read_table_str( cursor, 'CUSTDOC', 'CUST$D$DOCTYPE')
    read_columns_str( cursor, 'CUSTDOC', 'CUST$D$DOCTYPE')
    read_constr_c( cursor, 'CUSTDOC', 'CUST$D$DOCTYPE')
    read_constr_p( cursor, 'CUSTDOC', 'CUST$D$DOCTYPE')
    read_constr_fk( cursor, 'CUSTDOC', 'CUST$D$DOCTYPE')
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Помилка Oracle: {error_obj.code} - {error_obj.message}")
finally:
    if cursor:
        cursor.close()
        print("\nКурсор закрито.")
    if connection:
        connection.close()
        print("З'єднання з БД закрито.")

TABLE


,TAB_OWNER,TAB_NAME,TAB_TYPE,TAB_REM
0,CUSTDOC,CUST$D$DOCTYPE,TABLE,Types Customer Documents


TABLE COLUMS


,TAB_OWNER,TAB_NAME,COL_NAME,COL_TYPE,COL_LENGTH,COL_DEC,COL_REM
0,CUSTDOC,CUST$D$DOCTYPE,TYPEDOC,VARCHAR2,32,None,TYPE DOCUMET
1,CUSTDOC,CUST$D$DOCTYPE,NAMETYPE,VARCHAR2,64,None,None
2,CUSTDOC,CUST$D$DOCTYPE,IUSRNM,VARCHAR2,30,None,Who Inserted the record
3,CUSTDOC,CUST$D$DOCTYPE,IDT,DATE,7,None,Date and Time of Insertion
4,CUSTDOC,CUST$D$DOCTYPE,MUSRNM,VARCHAR2,30,None,Who Modified the record
5,CUSTDOC,CUST$D$DOCTYPE,MDT,DATE,7,None,Date and Time of Modification


TABLE CONSTRAINTS


,TAB_OWNER,TAB_NAME,CNST_NAME,CNST_VALUE
0,CUSTDOC,CUST$D$DOCTYPE,SYS_C008537,"""TYPEDOC"" IS NOT NULL"
1,CUSTDOC,CUST$D$DOCTYPE,CUST$D$DOCTYPE_NAMETYPE,NAMETYPE IS NOT NULL


TABLE PRIMARY KEY CONSTRAINT


,TAB_OWNER,TAB_NAME,CNST_NAME,CNST_TYPE,COL_NAME,COL_POS
0,CUSTDOC,CUST$D$DOCTYPE,CUST$D$DOCTYPE_PK,P,TYPEDOC,1


TABLE FORAINGN KEY CONSTRAINT
Запит не повернув жодного рядка.

Курсор закрито.
З'єднання з БД закрито.


## TALE ```CUSTDOC.CUST$DOCS``` 	

In [12]:
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення
connection = None
cursor = None
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення
try:
    connection = oracledb.connect(dsn=dsn)
    cursor = connection.cursor()
    read_table_str( cursor, 'CUSTDOC', 'CUST$DOCS')
    read_columns_str( cursor, 'CUSTDOC', 'CUST$DOCS')
    read_constr_c( cursor, 'CUSTDOC', 'CUST$DOCS')
    read_constr_p( cursor, 'CUSTDOC', 'CUST$DOCS')
    read_constr_fk( cursor, 'CUSTDOC', 'CUST$DOCS')
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Помилка Oracle: {error_obj.code} - {error_obj.message}")
finally:
    if cursor:
        cursor.close()
        print("\nКурсор закрито.")
    if connection:
        connection.close()
        print("З'єднання з БД закрито.")

TABLE


,TAB_OWNER,TAB_NAME,TAB_TYPE,TAB_REM
0,CUSTDOC,CUST$DOCS,TABLE,Customer Documents


TABLE COLUMS


,TAB_OWNER,TAB_NAME,COL_NAME,COL_TYPE,COL_LENGTH,COL_DEC,COL_REM
0,CUSTDOC,CUST$DOCS,IDDOC,NUMBER,22,0.0,Document Identifier
1,CUSTDOC,CUST$DOCS,CUSTID,VARCHAR2,9,NaN,Customer Identifier
2,CUSTDOC,CUST$DOCS,TYPEDOC,VARCHAR2,32,NaN,Type of Document
3,CUSTDOC,CUST$DOCS,ISACRUAL,VARCHAR2,1,NaN,"Migration Flag (Y-Not mirated, N-Migrated)"
4,CUSTDOC,CUST$DOCS,IUSRNM,VARCHAR2,30,NaN,Who Inserted the record
5,CUSTDOC,CUST$DOCS,IDT,DATE,7,NaN,Date and Time of Insertion
6,CUSTDOC,CUST$DOCS,MUSRNM,VARCHAR2,30,NaN,Who Modified the record
7,CUSTDOC,CUST$DOCS,MDT,DATE,7,NaN,Date and Time of Modification


TABLE CONSTRAINTS


...

TABLE PRIMARY KEY CONSTRAINT


,TAB_OWNER,TAB_NAME,CNST_NAME,CNST_TYPE,COL_NAME,COL_POS
0,CUSTDOC,CUST$DOCS,CUST$DOCS_PK,P,IDDOC,1


TABLE FORAINGN KEY CONSTRAINT


,TAB_OWNER,TAB_NAME,CNST_NAME,FK_COL_NAME,REF_TAB_OWNER,REF_TAB_NAME,FK_COL_NAME
0,CUSTDOC,CUST$DOCS,CUST$DOCS_FK_CUSTID,CUSTID,CUSTDOC,CUST$CUST,CUSTID
1,CUSTDOC,CUST$DOCS,CUST$DOCS_FK_IDDOC,TYPEDOC,CUSTDOC,CUST$D$DOCTYPE,TYPEDOC



Курсор закрито.
З'єднання з БД закрито.


## TABLE ```CUSTDOC.CUST$DOCS$ATTACH```

In [13]:
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення
connection = None
cursor = None
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення
try:
    connection = oracledb.connect(dsn=dsn)
    cursor = connection.cursor()
    read_table_str( cursor, 'CUSTDOC', 'CUST$DOCS$ATTACH')
    read_columns_str( cursor, 'CUSTDOC', 'CUST$DOCS$ATTACH')
    read_constr_c( cursor, 'CUSTDOC', 'CUST$DOCS$ATTACH')
    read_constr_p( cursor, 'CUSTDOC', 'CUST$DOCS$ATTACH')
    read_constr_fk( cursor, 'CUSTDOC', 'CUST$DOCS$ATTACH')
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Помилка Oracle: {error_obj.code} - {error_obj.message}")
finally:
    if cursor:
        cursor.close()
        print("\nКурсор закрито.")
    if connection:
        connection.close()
        print("З'єднання з БД закрито.")

TABLE


,TAB_OWNER,TAB_NAME,TAB_TYPE,TAB_REM
0,CUSTDOC,CUST$DOCS$ATTACH,TABLE,Customer Documents: Attached files


TABLE COLUMS


,TAB_OWNER,TAB_NAME,COL_NAME,COL_TYPE,COL_LENGTH,COL_DEC,COL_REM
0,CUSTDOC,CUST$DOCS$ATTACH,IDFL,NUMBER,22,0.0,File IDentifier
1,CUSTDOC,CUST$DOCS$ATTACH,IDDOC,NUMBER,22,0.0,Document IDentifier
2,CUSTDOC,CUST$DOCS$ATTACH,FILE_NAME,VARCHAR2,120,NaN,Document File name
3,CUSTDOC,CUST$DOCS$ATTACH,FILE_CONTENT,BLOB,4000,NaN,Document binary content
4,CUSTDOC,CUST$DOCS$ATTACH,FILE_DESC,VARCHAR2,120,NaN,Document description while uploading
5,CUSTDOC,CUST$DOCS$ATTACH,IUSRNM,VARCHAR2,30,NaN,Who Inserted the record
6,CUSTDOC,CUST$DOCS$ATTACH,IDT,DATE,7,NaN,Date and Time of Insertion
7,CUSTDOC,CUST$DOCS$ATTACH,MUSRNM,VARCHAR2,30,NaN,Who Modified the record
8,CUSTDOC,CUST$DOCS$ATTACH,MDT,DATE,7,NaN,Date and Time of Modification


TABLE CONSTRAINTS


...

TABLE PRIMARY KEY CONSTRAINT


,TAB_OWNER,TAB_NAME,CNST_NAME,CNST_TYPE,COL_NAME,COL_POS
0,CUSTDOC,CUST$DOCS$ATTACH,CUST$DOCS$ATTACH_PK,P,IDFL,1


TABLE FORAINGN KEY CONSTRAINT


,TAB_OWNER,TAB_NAME,CNST_NAME,FK_COL_NAME,REF_TAB_OWNER,REF_TAB_NAME,FK_COL_NAME
0,CUSTDOC,CUST$DOCS$ATTACH,CUST$DOCS$ATTACH_FK_IDDOC,IDDOC,CUSTDOC,CUST$DOCS,IDDOC



Курсор закрито.
З'єднання з БД закрито.


## TQBLE ```CUSTDOC.CUST$DOCS$URLS```

In [15]:
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення
connection = None
cursor = None
pd.set_option('display.max_columns', None)  # Відображати всі стовпці
pd.set_option('display.width', 3000)        # Збільшити ширину консолі для відображення
try:
    connection = oracledb.connect(dsn=dsn)
    cursor = connection.cursor()
    read_table_str( cursor, 'CUSTDOC', 'CUST$DOCS$URLS')
    read_columns_str( cursor, 'CUSTDOC', 'CUST$DOCS$URLS')
    read_constr_c( cursor, 'CUSTDOC', 'CUST$DOCS$URLS')
    read_constr_p( cursor, 'CUSTDOC', 'CUST$DOCS$URLS')
    read_constr_fk( cursor, 'CUSTDOC', 'CUST$DOCS$URLS')
except oracledb.Error as e:
    error_obj, = e.args
    print(f"Помилка Oracle: {error_obj.code} - {error_obj.message}")
finally:
    if cursor:
        cursor.close()
        print("\nКурсор закрито.")
    if connection:
        connection.close()
        print("З'єднання з БД закрито.")

TABLE


,TAB_OWNER,TAB_NAME,TAB_TYPE,TAB_REM
0,CUSTDOC,CUST$DOCS$URLS,TABLE,Migrated On Blob Storage Customer Documents


TABLE COLUMS


,TAB_OWNER,TAB_NAME,COL_NAME,COL_TYPE,COL_LENGTH,COL_DEC,COL_REM
0,CUSTDOC,CUST$DOCS$URLS,IDFL,NUMBER,22,0.0,File IDentifier
1,CUSTDOC,CUST$DOCS$URLS,IDDOC,NUMBER,22,0.0,Document IDentifier
2,CUSTDOC,CUST$DOCS$URLS,FILE_NAME,VARCHAR2,120,NaN,Document file name
3,CUSTDOC,CUST$DOCS$URLS,CONTAINER_NAME,VARCHAR2,120,NaN,Blob Storage Container Name
4,CUSTDOC,CUST$DOCS$URLS,CONTENT_TYPE,VARCHAR2,80,NaN,Document MIME Type
5,CUSTDOC,CUST$DOCS$URLS,FILE_SIZE,NUMBER,22,0.0,Size of document
6,CUSTDOC,CUST$DOCS$URLS,FILE_URL,VARCHAR2,240,NaN,File Blob Storage URL
7,CUSTDOC,CUST$DOCS$URLS,FILE_DESC,VARCHAR2,120,NaN,File Description
8,CUSTDOC,CUST$DOCS$URLS,IUSRNM,VARCHAR2,30,NaN,Who Inserted the record
9,CUSTDOC,CUST$DOCS$URLS,IDT,DATE,7,NaN,Date and Time of Insertion


TABLE CONSTRAINTS


...

TABLE PRIMARY KEY CONSTRAINT


,TAB_OWNER,TAB_NAME,CNST_NAME,CNST_TYPE,COL_NAME,COL_POS
0,CUSTDOC,CUST$DOCS$URLS,CUST$DOCS$URLS_PK,P,IDFL,1


TABLE FORAINGN KEY CONSTRAINT


,TAB_OWNER,TAB_NAME,CNST_NAME,FK_COL_NAME,REF_TAB_OWNER,REF_TAB_NAME,FK_COL_NAME
0,CUSTDOC,CUST$DOCS$URLS,CUST$DOCS$URLS_FK_IDDOC,IDDOC,CUSTDOC,CUST$DOCS,IDDOC



Курсор закрито.
З'єднання з БД закрито.
